In [ ]:
from exploitation_strategy import IceFishingExploitationStrategy
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt

In [ ]:
def simulate_spot_leaving(is_spot_successful=0):
    strategy = IceFishingExploitationStrategy(
                    baseline_weight=-3,
                    fish_catch_weight=-1.72,
                    time_weight=0.13,
                    social_feature_weight=0,
                    step_minutes=1/6
                )
    t_total = 0
    t_since_last_catch = 0
    stay = True
    
    while stay:
        t_total += 1
        t_since_last_catch += 1
        if not strategy.stay_on_patch(t_since_last_catch, is_spot_successful):
            stay = False
        
        # catch fish every 2 minutes
        if is_spot_successful and t_since_last_catch == 10*6:
            t_since_last_catch = 0
            
    return t_total

In [ ]:
time_unsuccessful_spots = [simulate_spot_leaving()*10 for _ in range(10000)]
time_successful_spots = [simulate_spot_leaving(is_spot_successful=1)*10 for _ in range(10000)]

In [ ]:
df = pd.DataFrame({
    "Angling Time spots (s)": time_unsuccessful_spots+time_successful_spots,
    "Is spot successful": ['No' for _ in range(10_000)] + ['Yes' for _ in range(10_000)]
})

In [ ]:
%matplotlib inline  
sns.catplot(
    data=df,
    #kind="boxen",
    kind="point",
    errorbar="sd",
    y="Angling Time spots (s)",
    x="Is spot successful",
    hue="Is spot successful",
    height=4
)



plt.show()